In [74]:
import datetime

from lxml import etree

In [75]:
doc_tree = etree.parse('assets/template.tcx')
doc_root = doc_tree.getroot()

In [76]:
nsmap = {'ns': doc_root.nsmap[None]}
act_elem = doc_root.find('./ns:Activities/ns:Activity', namespaces=nsmap)

In [77]:
trackpoint = doc_root.find('.//ns:Trackpoint', namespaces=nsmap)
tp_parent = trackpoint.getparent()

In [78]:
etree.strip_elements(doc_root, trackpoint.tag, with_tail=True)

In [79]:
grade_percent = -49
speed_ms = 1.0
n_t = 100

grade_dec = grade_percent / 100
timestep_sec = 10.0 / speed_ms
elev_0 = max(-speed_ms * timestep_sec * n_t * grade_dec, 0)

tp_parent.extend([
    etree.fromstring(
      f"""<Trackpoint>
        <Time>{(datetime.datetime(1989, 12, 4) + datetime.timedelta(seconds=time_index * timestep_sec)).strftime("%Y-%m-%dT%H:%M:%SZ")}</Time>
        <DistanceMeters>{(speed_ms * time_index * timestep_sec):.1f}</DistanceMeters>
        <AltitudeMeters>{(elev_0 + speed_ms * time_index * timestep_sec * grade_dec):.1f}</AltitudeMeters>
      </Trackpoint>"""
    )
    for time_index in range(n_t)
])

In [80]:
# Write a new file that can fool strava/trainingpeaks/garmin/goldencheetah.
doc_tree.write(
  f'out/synthetic_{"neg" if grade_percent < 0 else ""}{abs(grade_percent):.0f}_pct_{speed_ms:.0f}_ms_{n_t:.0f}_pts.tcx',
  pretty_print=True,
  xml_declaration=True,
  encoding='UTF-8'
)